### Libraries

In [61]:
import duckdb
from duckdb import DuckDBPyConnection

### Global Variables

In [62]:
DUCK_DB: str = "include/database/duck.db"
STATIONS_DDL: str = "include/sql/weather/stations_table_ddl.sql"
WEATHER_OBS_DDL: str = "include/sql/weather/weather_obs_table_ddl.sql"
AVG_TEMP_LAST_WEEK: str = "include/sql/weather/avg_temp_last_week.sql"
MAX_WIND_CHANGE_LAST_WEEK: str = "include/sql/weather/max_wind_change_last_week.sql"

### Dabase

#### Create database

The following command will create the database file, make sure that after you run it you have the file `duck.db` under this path:
*  `include/database`

When that command run for the first time, if the file doesn't exist will create a new one. If exist it will connect to that file.

In [63]:
con: DuckDBPyConnection = duckdb.connect(DUCK_DB)

### DDL

#### stations table

In [68]:
with duckdb.connect(DUCK_DB) as con:
    with open(STATIONS_DDL) as file:
        sql_query: str = file.read()
        print(f"Executing query: \n {sql_query}")
        con.query(query=sql_query)
        print("Done :)")

Executing query: 
 CREATE OR REPLACE TABLE stations (
    station_id VARCHAR PRIMARY KEY,
    station_name VARCHAR,
    station_timezone VARCHAR
);

Done :)


#### weather_obs table

In [69]:
with duckdb.connect(DUCK_DB) as con:
    with open(WEATHER_OBS_DDL) as file:
        sql_query: str = file.read()
        print(f"Executing query: \n {sql_query}")
        con.query(query=sql_query)
        print("Done :)")

Executing query: 
 CREATE OR REPLACE TABLE weather_obs (
    station_id VARCHAR,
    latitude DOUBLE,
    longitude DOUBLE,
    observation_timestamp TIMESTAMP,
    temperature DOUBLE,
    wind_speed DOUBLE,
    humidity DOUBLE
);

Done :)


### Analytic

#### Average temp last 7 days

In [41]:
with duckdb.connect(DUCK_DB) as con:
    with open(AVG_TEMP_LAST_WEEK) as file:
        sql_query: str = file.read()
        print(f"Executing query: \n {sql_query}")
        result = con.query(query=sql_query)
        print("Result of the sql query: ")
        print(result)

Executing query: 
 SELECT
    stations.station_id,
    station_name,
    AVG(temperature) AS average_temperature
FROM
    weather_obs
INNER JOIN
    stations
ON weather_obs.station_id = stations.station_id
WHERE
    observation_timestamp BETWEEN (current_date() - INTERVAL 7 DAY) AND current_date()
GROUP BY
    1,2;
Result of the sql query: 
┌────────────┬───────────────────────┬─────────────────────┐
│ station_id │     station_name      │ average_temperature │
│  varchar   │        varchar        │       double        │
├────────────┼───────────────────────┼─────────────────────┤
│ 0112W      │ Lafayette High School │  26.461569965870307 │
└────────────┴───────────────────────┴─────────────────────┘



#### Maximum wind speed change

In [70]:
with duckdb.connect(DUCK_DB) as con:
    with open(MAX_WIND_CHANGE_LAST_WEEK) as file:
        sql_query: str = file.read()
        print(f"Executing query: \n {sql_query}")
        result = con.query(query=sql_query)
        print("Result of the sql query: ")
        print(result)

Executing query: 
 WITH lagged_data AS (
  SELECT
    stations.station_id,
    station_name,
    wind_speed,
    LAG(wind_speed) OVER (ORDER BY observation_timestamp) AS previous_wind_speed
  FROM
    weather_obs
  INNER JOIN
    stations
  ON weather_obs.station_id = stations.station_id
  WHERE
    observation_timestamp BETWEEN (CURRENT_DATE() - INTERVAL 7 DAY) AND CURRENT_DATE()
)
SELECT
  station_id,
  station_name,
  ROUND(MAX(ABS((wind_speed - previous_wind_speed))), 2) AS max_wind_speed_change
FROM
  lagged_data
GROUP BY 1,2;

Result of the sql query: 
┌────────────┬──────────────┬───────────────────────┐
│ station_id │ station_name │ max_wind_speed_change │
│  varchar   │   varchar    │        double         │
├────────────┴──────────────┴───────────────────────┤
│                      0 rows                       │
└───────────────────────────────────────────────────┘

